In [1]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.calibration import CalibrationDisplay
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay,DetCurveDisplay,PrecisionRecallDisplay,RocCurveDisplay,classification_report,r2_score
from sklearn.model_selection import GridSearchCV,KFold,cross_validate,train_test_split
from df_after_transform import df_after_transform
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
set_config(display="diagram")

In [2]:
bod_df = pd.read_csv('./input_data_clean/BOD_DF.csv')


In [3]:
bod_df.describe()

,Unnamed: 0,gvkey,total_sec,othcomp,noneq_incent,cash_fees,stock_awards,option_awards,year,total_director_comp,...,total_option_awards,mkvalt,liquidity,net_income,num_employees,market_value,debt_to_equity,assets_in_place,capex_by_assets,return_on_equity
count,8773.000000,8773.000000,8.773000e+03,8773.000000,8773.000000,8773.000000,8.773000e+03,8773.000000,8773.000000,8773.000000,...,8773.000000,6.922000e+03,3885.000000,3149.000000,8699.000000,6829.000000,8757.000000,7321.000000,3147.000000,3149.000000
mean,4386.000000,51126.038983,4.872646e+02,17.904714,0.028434,100.167401,3.556027e+02,13.585409,2015.505414,2226.621567,...,211.805984,4.102918e+04,0.137121,1678.147851,40.622200,33201.705902,3.789363,0.246035,0.045511,0.066440
std,2532.691289,63506.010925,2.057758e+04,180.524592,1.063338,62.210634,2.057780e+04,117.750978,2.300127,1449.372677,...,648.630627,7.221464e+04,0.163929,3927.863067,97.706821,56756.406571,20.894526,0.350103,0.040322,0.080396
min,0.000000,1045.000000,0.000000e+00,-252.000000,0.000000,0.000000,0.000000e+00,0.000000,2011.000000,0.000000,...,0.000000,6.289170e+01,-0.394751,-23119.000000,0.052000,62.891700,-855.695652,0.000000,0.000000,-1.226993
25%,2193.000000,6781.000000,1.999150e+02,0.000000,0.000000,72.000000,8.999700e+01,0.000000,2014.000000,1334.310000,...,0.000000,1.012891e+04,0.018937,244.100000,4.058500,7878.348200,1.076222,0.006695,0.018492,0.030410
50%,4386.000000,14824.000000,2.555000e+02,0.000000,0.000000,100.000000,1.300400e+02,0.000000,2016.000000,1992.099000,...,0.000000,1.790603e+04,0.096980,593.804000,13.400000,15235.260000,2.190500,0.077511,0.033888,0.063631
75%,6579.000000,65609.000000,3.065310e+02,5.000000,0.000000,125.000000,1.749180e+02,0.000000,2018.000000,2803.750000,...,122.400000,3.633528e+04,0.234610,1574.000000,40.000000,29901.693000,6.514269,0.326998,0.060842,0.103170
max,8772.000000,316056.000000,1.927511e+06,7688.890000,78.000000,941.186000,1.927511e+06,6838.600000,2019.000000,24450.872000,...,11948.963000,1.073391e+06,0.768114,53394.000000,2300.000000,790050.098100,990.205128,4.979567,0.467642,0.398172


In [4]:
bod_df.drop('Unnamed: 0', axis = 1)

,gvkey,ticker,total_sec,othcomp,noneq_incent,cash_fees,stock_awards,option_awards,spcode,cusip,...,size_category,signature_index,liquidity,net_income,num_employees,market_value,debt_to_equity,assets_in_place,capex_by_assets,return_on_equity
0,1045,AAL,47.951,0.951,0.0,47.000,0.000,0.0,SP,02376R10,...,Tiny,589b2d914f25e0a1ff8f707f7aa52096,-0.077408,-471.0,78.25,2597.5755,-7.359442,1.064931,0.078205,-0.018774
1,1045,AAL,0.000,0.000,0.0,0.000,0.000,0.0,SP,02376R10,...,Medium,436a3154f7acadd68f77107364944857,-0.078539,-1979.0,80.10,117.3438,-4.353677,1.042184,0.067511,-0.082984
2,1045,AAL,418.608,36.108,0.0,102.500,280.000,0.0,SP,02376R10,...,Big,dc94df973b47ec03bf615d54bd90e720,-0.094938,-1876.0,77.75,266.5571,-3.943533,1.030753,0.080306,-0.079796
3,1045,AAL,286.838,29.338,0.0,117.500,140.000,0.0,SP,02376R10,...,Big,c2357bce3ae6160bca4e3155a24b1a0a,0.012229,-1834.0,110.40,6591.9923,-16.480776,0.718861,0.073655,-0.043380
4,1045,AAL,317.368,32.368,0.0,135.000,150.000,0.0,SP,02376R10,...,Big,7abb44613cbb465cef913dbfd121155a,-0.030225,2882.0,113.30,37405.5843,20.658090,0.807452,0.121336,0.065843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768,316056,ALLE,290.810,20.754,0.0,200.000,70.056,0.0,SP,G0176J10,...,Medium,9a2552c4e1a18391ae83d6a74537a7b6,0.218546,31.0,8.00,4243.5215,-23.451613,0.287489,0.010203,0.015657
8769,316056,ALLE,236.508,0.000,0.0,166.500,70.008,0.0,SP,G0176J10,...,Medium,c5aa2527abdb80465777f4eeb1bce3fc,0.219505,175.2,8.50,5314.7873,-416.125000,0.291086,0.025547,0.086909
8770,316056,ALLE,44.511,0.000,0.0,44.511,0.000,0.0,SP,G0176J10,...,Medium,b33c9610b47875a2cebb9d47b97dd758,0.126023,153.9,9.40,6327.7267,88.109375,0.269768,0.015403,0.067343
8771,316056,ALLE,240.033,0.000,0.0,140.000,100.033,0.0,SP,G0176J10,...,Medium,46be5701765385ac27f8a4d0adabf5f6,0.177850,229.1,9.40,6097.5360,18.808473,0.284951,0.018911,0.101940


In [5]:
bod_df.to_csv('./input_data_clean/BOD_DF.csv')

In [7]:
bod_df = pd.read_csv('./Input_data/DirectorComp.csv')
bod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49247 entries, 0 to 49246
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GVKEY          49247 non-null  int64  
 1   DIRNBR         49247 non-null  int64  
 2   DIRNAME        49247 non-null  object 
 3   CASH_FEES      49247 non-null  float64
 4   STOCK_AWARDS   49247 non-null  float64
 5   OPTION_AWARDS  49247 non-null  float64
 6   NONEQ_INCENT   49247 non-null  float64
 7   PENSION_CHG    49247 non-null  float64
 8   OTHCOMP        49247 non-null  float64
 9   TOTAL_SEC      49247 non-null  float64
 10  YEAR           49247 non-null  int64  
 11  CONAME         49247 non-null  object 
 12  CUSIP          49247 non-null  object 
 13  EXCHANGE       49247 non-null  object 
 14  ADDRESS        49247 non-null  object 
 15  CITY           49247 non-null  object 
 16  STATE          47165 non-null  object 
 17  ZIP            49134 non-null  object 
 18  SICDES